In [95]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import pathlib

from credentials import SAMPLE_DB, FULL_DB
from librarian import SAMPLE_DATA_DIR

import encoders
import db_connect
import helpers

In [96]:
DATA_DIR = SAMPLE_DATA_DIR
CHUNK_SIZE = 128
LAYER_UNITS = max(1, CHUNK_SIZE // 10)

INFER_TEXT = False
INFER_KEY = not INFER_TEXT

if INFER_TEXT:
    MAIN_ACCURACY_METRIC = "mae"
    GOOD_ACCURACY_DIRECTION = "min" # Some accuracy metrics go up for better results, some go down
    LOSS_METRIC = "mean_squared_error"
    OUTPUT_SIZE = CHUNK_SIZE
else:
    MAIN_ACCURACY_METRIC = "mae"
    GOOD_ACCURACY_DIRECTION = "min" # Some accuracy metrics go up for better results, some go down
    LOSS_METRIC = "mae"
    OUTPUT_SIZE = 1 # actually depends on cipher

ENCRYPTED_FILE_LIMIT = 30 # -1 to disable limit
SPLIT_SEED = 42
SPLIT_TEST_SIZE = 0.25     # 0.25 is default

LOAD_BEST_MODEL = False # If False, a new model will be created from scratch
SAVE_BEST_MODEL = False
TRAIN_MODEL = True
EPOCHS = 5

db = db_connect.DB(SAMPLE_DB)

In [97]:
# Get database IDs for encoders and key types

encoder_ids= {}
key_type_ids = {}

with db.get_session() as session:
    for encoder in encoders.ALL_ENCODER_NAMES:
        id = db.get_encoder_id(session, encoder)
        encoder_ids[encoder] = id

    print(f"Encoder IDs: {encoder_ids}")

    for key_type in encoders.KEY_NAMES:
        id = db.get_key_type_id(session, key_type)
        key_type_ids[key_type] = id

    print(f"Key Type IDs: {key_type_ids}")

Encoder IDs: {'None': 1, 'Simplifier': 2, 'Caesar Cipher': 3, 'Substitution Cipher': 4, 'Enigma Machine': 5}
Key Type IDs: {'Character Offset': 1, 'Character Map': 2, 'Rotor Settings': 3}


In [98]:
# Map source ID to plaintext file (1) details, and source ID to corresponding ciphertext files (1+) details
sid_to_p = {}
sid_to_c = {}

cipher_id = encoder_ids[encoders.ENCODER_CAESAR]
with db.get_session() as session:
    # Get all files encrypted with the cipher we care about
    encrypted_files = db.get_files_by_source_and_encoder(session, -1, cipher_id)

    if len(encrypted_files) > ENCRYPTED_FILE_LIMIT and ENCRYPTED_FILE_LIMIT > 0:
        encrypted_files = encrypted_files[0:ENCRYPTED_FILE_LIMIT]

    for c in encrypted_files:
        sid = c.source_id
    
        if sid not in sid_to_p:
            plaintext_ids = db.get_files_by_source_and_encoder(session, sid, encoder_ids[encoders.ENCODER_SIMPLIFIER])
            if len(plaintext_ids) != 1:
                raise Exception(f"Found {len(plaintext_ids)} plaintexts for source ID {sid}; should be exactly 1")
            sid_to_p[sid] = plaintext_ids[0]

        if sid not in sid_to_c:
            sid_to_c[sid] = []
        sid_to_c[sid].append(c)

len(sid_to_p), len(sid_to_c)

(6, 6)

In [99]:
# Build up the features (X, the cipher texts as values) and targets (y, either the plain texts as values OR the key).
# Note targets are not necessarily unique.
X = []
y = []

with db.get_session() as session:
    for sid in sid_to_p:
        if INFER_TEXT:
            plaintext = helpers.read_text_file(sid_to_p[sid].path)
            target_chunks = helpers.string_to_bytes(plaintext, CHUNK_SIZE)    
    
        for c in sid_to_c[sid]:
            ciphertext = helpers.read_text_file(c.path)
            feature_chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)

            if INFER_KEY:
                key_value = float(db.get_key_by_id(session, c.key_id).value)
    
            for i in range (len(feature_chunks)):
                X.append(feature_chunks[i])

                if INFER_TEXT:
                    y.append(target_chunks[i])                

                if INFER_KEY:
                    y.append(key_value)

X = np.array(X)
y = np.array(y)

X.shape, y.shape, X[0].shape, y[0].shape, X[0], y[0]

((99444, 128),
 (99444,),
 (128,),
 (),
 array([41., 41., 65., 67., 10., 59., 70., 93., 39., 59., 40., 91., 74.,
        40., 34., 67., 39., 34., 40., 72., 39., 39., 60., 68., 33., 40.,
        63., 43., 67., 74., 40., 63., 39., 39., 44., 43., 32., 40., 43.,
        32., 59., 40., 69., 44., 39., 40., 10., 32., 62., 46., 32., 39.,
        41., 59., 46., 68., 69., 67., 46., 91., 70., 69., 39., 59., 40.,
        65., 67., 10., 10., 58., 67., 39., 43., 59., 46., 32., 34., 40.,
        69., 39., 43., 63., 40., 43., 69., 40., 44., 69., 69., 65., 61.,
        36., 36., 72., 72., 72., 35., 65., 34., 59., 65., 35., 32., 39.,
        69., 41., 41., 40., 69., 44., 39., 40., 67., 43., 69., 40., 67.,
        43., 93., 39., 41., 41., 40., 91., 74., 40., 47., 43.]),
 47.0)

In [100]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SPLIT_TEST_SIZE, random_state=SPLIT_SEED)
print( len(X), len(y), len(X_train), len(X_test), len(y_train), len(y_test) )

99444 99444 74583 24861 74583 24861


In [101]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((74583, 128), (24861, 128))

In [102]:
if LOAD_BEST_MODEL:
    print(f"Loading model from {BEST_PATH}")
    nn = tf.keras.models.load_model(BEST_PATH)
else:
    print("Building new model")
    in_shape = (CHUNK_SIZE,)
    nn = tf.keras.models.Sequential()
    
    # Input layer
    nn.add(tf.keras.Input(shape=in_shape))
    
    # Hidden layers
    activations = ["tanh", "relu", "elu", "exponential", "gelu", "mish", "relu6", "tanh", "selu"]
    unit_counts = [LAYER_UNITS]
    for u in unit_counts:
        for a in activations:
            nn.add(tf.keras.layers.Dense(units=u, activation=a))
    
    # Output layer
    nn.add(tf.keras.layers.Dense(units=OUTPUT_SIZE))
    
# Check the structure of the model
nn.summary()

Building new model


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_70 (Dense)                │ (None, 12)             │         1,548 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 12)             │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_72 (Dense)                │ (None, 12)             │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_73 (Dense)                │ (None, 12)             │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_74 (Dense)                │ (None, 12)             │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_75 (Dense)                │ (None, 12)             │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_76 (Dense)                │ (None, 12)             │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_77 (Dense)                │ (None, 12)             │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_78 (Dense)                │ (None, 12)             │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_79 (Dense)                │ (None, 1)              │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,809 (10.97 KB)

 Trainable params: 2,809 (10.97 KB)

 Non-trainable params: 0 (0.00 B)

In [103]:
%%time

# Set up training checkpoint to save after each epoch, if it is a new best model:
BEST_PATH = './best.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=BEST_PATH,
    monitor=MAIN_ACCURACY_METRIC,
    mode=GOOD_ACCURACY_DIRECTION,
    save_best_only=True,
    save_weights_only=False,
    verbose=1)

if TRAIN_MODEL:
    print(f"Training model")
    
    # Compile the Sequential model together and customize metrics
    nn.compile(loss=LOSS_METRIC, optimizer="sgd", metrics=[MAIN_ACCURACY_METRIC])
    
    # Fit the model to the training data
    callbacks = None
    if SAVE_BEST_MODEL:
        callbacks = [model_checkpoint_callback]
    fit_model = nn.fit(X_train_scaled, y_train, epochs=EPOCHS, callbacks=callbacks)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Training model
Epoch 1/5
2331/2331 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 15.4087 - mae: 15.4087
Epoch 2/5
2331/2331 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 11.7122 - mae: 11.7122
Epoch 3/5
2331/2331 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 11.2459 - mae: 11.2459
Epoch 4/5
2331/2331 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 10.2529 - mae: 10.2529
Epoch 5/5
2331/2331 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 9.8985 - mae: 9.8985
777/777 - 3s - 3ms/step - loss: 9.7261 - mae: 9.7261
Loss: 9.726078987121582, Accuracy: 9.726078987121582
CPU times: user 44.6 s, sys: 12.6 s, total: 57.2 s
Wall time: 46.7 s


In [ ]:
def decode_chunks_with_model(chunks: list[list], model, scaler, input_already_scaled = True) -> list[list]:
    if input_already_scaled:
        return model.predict(chunks)
    else:
        return model.predict(scaler.transform(chunks))

def decode_text_with_model(ciphertext: str, model, scaler) -> str:
    chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)
    decoded_chunks = decode_chunks_with_model(chunks, model, scaler, input_already_scaled = False)
    return helpers.bytes_to_string(decoded_chunks)

def infer_key_with_model(ciphertext: str, model, scaler) -> int:
    chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)
    key = int(round(model.predict(scaler.transform(chunks))[0][0]))
    return key

cipher_file_db = sid_to_c[list(sid_to_c.keys())[0]][0]
ciphertext_path = cipher_file_db.path
ciphertext = helpers.read_text_file(ciphertext_path)
ciphertext = ciphertext[0:CHUNK_SIZE*2]
    
if INFER_TEXT:    
    print("Decoded   : ", decode_text_with_model(ciphertext, nn, X_scaler))
if INFER_KEY:
    with db.get_session() as session:
        correct_key = int(db.get_key_by_id(session, cipher_file_db.key_id).value)
    print("Correct Key: ", correct_key)
    
    inferred_key = infer_key_with_model(ciphertext, nn, X_scaler)
    print("Inferred Key: ", inferred_key)

list(sid_to_c.keys())[0]